# How to Run this Notebook in the ros:indigo docker container
## Run the Docker Container
### Do port forwarding.

docker run -it --name=ros_jupyter -v ~/Downloads/Didi-Training-Release-1:/home/didi <strong>-p 8888:8888</strong> ros:indigo

### Add a volume pointed to the directory with the training data.

This is the parameter and argument to use to do that: -v host_directory:container_directory

docker run -it --name=ros_jupyter <strong>-v ~/Downloads/Didi-Training-Release-1:/home/didi</strong> -p 8888:8888 ros:indigo

### Install Jupyter in the Docker Container

This is how I did it. 
1. Install [pip](http://stackoverflow.com/questions/36611052/install-pip-in-docker)

```
sudo apt-get update
apt-get install python-pip
pip install --upgrade pip
```

2. Install jupyter with pip 

```sudo pip install jupyter```

### Run the Jupyter Notebook in the docker container.

The jupyter notebook must be [running on 0.0.0.0](http://stackoverflow.com/questions/38830610/access-jupyter-notebook-running-on-docker-container) instead of local host.

```jupyter notebook --ip 0.0.0.0```

Then you can go ahead and open it by copy and pasting the url or ctrl+clicking it.
![Jupyter-IP](./resources/jupyter_ip.png)

# Using rosbag
First let's look at the API example from https://github.com/bostondiditeam/ros/blob/master/docs/rosbag.md

Example usage for write:

In [97]:
import rosbag
from std_msgs.msg import Int32, String

bag = rosbag.Bag('test.bag', 'w')

try:
    str = String()
    str.data = 'foo'

    i = Int32()
    i.data = 42

    bag.write('chatter', str)
    bag.write('numbers', i)
finally:
    bag.close()

Example usage for read:

In [98]:
bag = rosbag.Bag('test.bag')
for topic, msg, t in bag.read_messages(topics=['chatter', 'numbers']):
    print msg
bag.close()

data: foo
data: 42


## Look at the bags


In [13]:
import os

# I set up a volume in the docker container that points to the data on my host machine.
os.listdir("/home/didi/")

['NOISY_spin_2.bag',
 'spin_3.bag',
 'approach_1.bag',
 '.ipynb_checkpoints',
 'Noisy',
 '5mph.bag',
 'overtake.bag',
 'intersection_1.bag',
 'test.bag',
 'corner_pass.bag',
 'spin.bag',
 'approach_3.bag',
 'Untitled.ipynb',
 'spin_shoreline.bag',
 'approach_2.bag',
 'sitting.bag',
 'README.md']

Notice that 'test.bag' is included in the list.  I'm also running the Jupyter notebook in this volume for convenience.

### Check out a sample from each bag
Loading all the data from each bag wouldn't be practical.

In [15]:
bag = rosbag.Bag('NOISY_spin_2.bag') # TAB Completion works in Jupyter for the filenames.

#### Jupyter Tip - Use Tab and Shift-Tab a lot
Tab will open a suggestion list or autocomplete a symbol.
![Tab-Suggestions](./resources/jupyter_tip_1.png)

In [42]:
# Let's see what topics we have available to us in the NOISY_spin_2.bag
info = bag.get_type_and_topic_info()
info.topics.keys()

['/diagnostics_toplevel_state',
 '/gps/rtkfix',
 '/cloud_nodelet/parameter_descriptions',
 '/obs1/gps/rtkfix',
 '/gps/time',
 '/image_raw',
 '/radar/points',
 '/rosout',
 '/obs1/gps/time',
 '/tf',
 '/velodyne_points',
 '/velodyne_packets',
 '/velodyne_nodelet_manager/bond',
 '/obs1/gps/fix',
 '/gps/fix',
 '/radar/tracks',
 '/diagnostics',
 '/cloud_nodelet/parameter_updates',
 '/radar/range',
 '/diagnostics_agg']

In [52]:
# Let's take a closer look at '/obs1/gps/rtkfix'
info.topics["/obs1/gps/rtkfix"]

TopicTuple(msg_type='nav_msgs/Odometry', message_count=3050, connections=1, frequency=10.009388192450785)

In [94]:
# rosbag.Bag.read_messages() creates a generator that yields a rosbag.bag.BagMessage
bag_reader = bag.read_messages(topics=['/obs1/gps/rtkfix'])
sample_message = bag_reader.next()
print "Type:", type(sample_message)
print "Message:\n",sample_message

Type: <class 'rosbag.bag.BagMessage'>
Message:
BagMessage(topic='/obs1/gps/rtkfix', message=header: 
  seq: 14758
  stamp: 
    secs: 1490149914
    nsecs: 857044240
  frame_id: gps
child_frame_id: ''
pose: 
  pose: 
    position: 
      x: -7.327
      y: -15.881
      z: -0.051
    orientation: 
      x: 9.47027447621e-05
      y: -4.7351372381e-05
      z: 0.000994378820002
      w: 0.0
  covariance: [196000000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 196000000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1368999936.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1000.0]
twist: 
  twist: 
    linear: 
      x: 0.002
      y: -0.001
      z: 0.021
    angular: 
      x: 0.0
      y: 0.0
      z: 0.0
  covariance: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1000.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1000.0], timestamp=rospy.Time[1490149914877343408])


In [95]:
# You can access all of the above data with dot notation
sample_message.message.pose.pose.position.x

-7.327